In [10]:
import pandas as pd
import datetime
from pathlib import Path
import matplotlib.pyplot as plt

from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate

from sklearn.model_selection import train_test_split as  train_test_split_sklearn

df = pd.read_csv("data/user-item-interaction.csv")

In [11]:
print(df.shape)
df.tail()

(2633521, 8)


,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
2633516,2020-11-21 10:10:01 UTC,2388440981134693942,1515966223526602848,2.268105e+18,electronics.smartphone,oppo,138.87,1.515916e+18
2633517,2020-11-21 10:10:13 UTC,2388440981134693943,1515966223509089282,2.268105e+18,electronics.smartphone,apple,418.96,1.515916e+18
2633518,2020-11-21 10:10:30 UTC,2388440981134693944,1515966223509089917,2.268105e+18,appliances.personal.scales,vitek,12.48,1.515916e+18
2633519,2020-11-21 10:10:30 UTC,2388440981134693944,2273948184839454837,2.268105e+18,NaN,moulinex,41.64,1.515916e+18
2633520,2020-11-21 10:10:30 UTC,2388440981134693944,1515966223509127566,2.268105e+18,appliances.kitchen.blender,redmond,53.22,1.515916e+18


In [15]:
print(df.order_id.duplicated().sum())
df.drop_duplicates('order_id', inplace=True)

0


In [19]:
df.dropna(
    subset=['event_time', 'order_id', 'product_id', 'category_id', 'price', 'user_id'],
    inplace=True
)
df.dtypes

event_time        object
order_id           int64
product_id         int64
category_id      float64
category_code     object
brand             object
price            float64
user_id          float64
dtype: object

In [22]:
#Split data into train and test

train = df.sample(frac = 0.7)
test = df.drop(train.index)

print(df.shape)
print(train.shape)
print(test.shape)

(389709, 8)
(272796, 8)
(116913, 8)


In [42]:
test_products = test['product_id']


df_with_test_products = df[df['product_id'].isin(test_products.tolist()) == True]
df_with_test_products = df_with_test_products.sort_values(by = 'event_time').reset_index(drop=True)

In [46]:
df_valid = (df_with_test_products[['product_id','user_id']]).copy()
df_valid = df_valid.groupby(df_valid.columns.tolist(), as_index=False).size()
df_valid.rename(columns={'size': 'rating'}, inplace=True)
df_valid['rating'] = df_valid['rating'].apply(lambda x: x+1 if x < 3 else 4)

df_valid #['count'].describe()

,product_id,user_id,rating
0,1515966223509088493,1.515916e+18,2
1,1515966223509088493,1.515916e+18,2
2,1515966223509088493,1.515916e+18,2
3,1515966223509088493,1.515916e+18,2
4,1515966223509088493,1.515916e+18,2
...,...,...,...
327025,2388434452476864413,1.515916e+18,2
327026,2388434452476864505,1.515916e+18,2
327027,2388434452476875362,1.515916e+18,2
327028,2388434452476875362,1.515916e+18,2


In [ ]:
svd = SVD()
data_tuning = build_data_set(df_train)

reader = Reader(rating_scale=(1, 4))
data = Dataset.load_from_df(data_tuning, reader)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
trainset = data.build_full_trainset()
svd.fit(trainset)